In [1]:
import pandas as pd
import torch.optim as optim
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

import math
import time
import pandas as pd
import numpy as np
import random
from typing import Tuple
import torch
import torch.nn as nn
import torch.optim as optim
import string
import random
from collections import Counter
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader


# LOADING DATA FROM CSV FILES

In [2]:
def load_data(path):
    data = pd.read_csv(path, header=None, names=["en", "ma", ""], skip_blank_lines=True)
    data = data.dropna(subset=['en', 'ma'])
    return data[['en', 'ma']]


In [3]:
train = load_data("/kaggle/input/marathi-dataset/mar/mar_train.csv")
val = load_data("/kaggle/input/marathi-dataset/mar/mar_valid.csv")
test = load_data("/kaggle/input/marathi-dataset/mar/mar_test.csv")



#TOKENIZE DATASET

In [4]:
def unique_tokenize(data):
    english_tokens = set(''.join(data['en']))
    marathi_tokens = set(''.join(data['ma']))
    return sorted(list(marathi_tokens)), sorted(list(english_tokens))


marathi_tokens , english_tokens = unique_tokenize(train)


In [5]:
def tokenize_map(hindi_tokens, english_tokens):
    # Create token maps for English and Marathi
    english_token_map = {token: idx + 1 for idx, token in enumerate(english_tokens)}
    marathi_token_map = {token: idx + 1 for idx, token in enumerate(hindi_tokens)}

    # Create reverse token map for Marathi
    reverse_marathi_token_map = {idx + 1: token for idx, token in enumerate(hindi_tokens)}

    
    marathi_token_map[" "] = 0
    english_token_map[" "] = 0
    reverse_marathi_token_map[65] = ';'
    marathi_token_map[';'] = 65
    reverse_marathi_token_map[0] = ''
    marathi_token_map['<unk>'] = 64
    english_token_map[';'] = 27
    reverse_marathi_token_map[64] = '<unk>'
    reverse_marathi_token_map[66] = '.'
    english_token_map['.'] = 28
    marathi_token_map['.'] = 66


    return marathi_token_map, english_token_map, reverse_marathi_token_map
mar_token_map, eng_token_map,reverse_marathi_token_map = tokenize_map(marathi_tokens , english_tokens)

print(mar_token_map)
print(reverse_marathi_token_map)
print(eng_token_map)

print('mar:',len(mar_token_map))
print('eng:',len(eng_token_map))

{'ँ': 1, 'ं': 2, 'ः': 3, 'अ': 4, 'आ': 5, 'इ': 6, 'ई': 7, 'उ': 8, 'ऊ': 9, 'ऋ': 10, 'ऍ': 11, 'ए': 12, 'ऐ': 13, 'ऑ': 14, 'ओ': 15, 'औ': 16, 'क': 17, 'ख': 18, 'ग': 19, 'घ': 20, 'च': 21, 'छ': 22, 'ज': 23, 'झ': 24, 'ञ': 25, 'ट': 26, 'ठ': 27, 'ड': 28, 'ढ': 29, 'ण': 30, 'त': 31, 'थ': 32, 'द': 33, 'ध': 34, 'न': 35, 'प': 36, 'फ': 37, 'ब': 38, 'भ': 39, 'म': 40, 'य': 41, 'र': 42, 'ल': 43, 'ळ': 44, 'व': 45, 'श': 46, 'ष': 47, 'स': 48, 'ह': 49, '़': 50, 'ा': 51, 'ि': 52, 'ी': 53, 'ु': 54, 'ू': 55, 'ृ': 56, 'ॅ': 57, 'े': 58, 'ै': 59, 'ॉ': 60, 'ो': 61, 'ौ': 62, '्': 63, ' ': 0, ';': 65, '<unk>': 64, '.': 66}
{1: 'ँ', 2: 'ं', 3: 'ः', 4: 'अ', 5: 'आ', 6: 'इ', 7: 'ई', 8: 'उ', 9: 'ऊ', 10: 'ऋ', 11: 'ऍ', 12: 'ए', 13: 'ऐ', 14: 'ऑ', 15: 'ओ', 16: 'औ', 17: 'क', 18: 'ख', 19: 'ग', 20: 'घ', 21: 'च', 22: 'छ', 23: 'ज', 24: 'झ', 25: 'ञ', 26: 'ट', 27: 'ठ', 28: 'ड', 29: 'ढ', 30: 'ण', 31: 'त', 32: 'थ', 33: 'द', 34: 'ध', 35: 'न', 36: 'प', 37: 'फ', 38: 'ब', 39: 'भ', 40: 'म', 41: 'य', 42: 'र', 43: 'ल', 44: 'ळ', 45: 'व', 46: '

#MAXIMUM WORD LENGTH THAT ARE PRESENT IN THE DATASET




In [6]:

x = test['en'].values
y = test['ma'].values

#Getting max length
max_eng_len = max([len(i) for i in x]) + 2
max_mar_len = max([len(i) for i in y]) + 2
print(max_eng_len,max_mar_len)

30 22


# ONE HOT ENCODING/EMBEDDING OUR INPUT



In [7]:

import torch
#unknown token present in validation set as 'r.(in marathi)'
unknown_token=64
def process(data):
    x,y = data['en'].values, data['ma'].values
    x = ";" + x + "."
    y = ";" + y + "."
    
    
    a = torch.zeros((len(x),max_eng_len),dtype=torch.int64)
    print(a.shape)
    
    b = torch.zeros((len(y),max_eng_len),dtype=torch.int64)
    
    data=[]
    for i,(xx,yy) in enumerate(zip(x,y)):
        for j,ch in enumerate(xx):
            a[i,j] = eng_token_map[ch]

        #a[i,j+1:] = eng_token_map[" "]
        for j,ch in enumerate(yy):
            if ch in mar_token_map: 
             b[i,j] = mar_token_map[ch]
            else:
              b[i,j]= unknown_token


    data = [(a[i], b[i]) for i in range(len(x))]
    print(a.shape)
    print(b.shape)
    return data

train_process=process(train)
val_process=process(val)
test_process=process(test)


torch.Size([51200, 30])
torch.Size([51200, 30])
torch.Size([51200, 30])
torch.Size([4096, 30])
torch.Size([4096, 30])
torch.Size([4096, 30])
torch.Size([4096, 30])
torch.Size([4096, 30])
torch.Size([4096, 30])


In [8]:
#Used later for reading the words

from torch import tensor
def reverse_tokenize(data):
  data=[int(i) for i in data]
  predicted_seq = [reverse_marathi_token_map[idx] for idx in data]
  predicted_seq=''.join(predicted_seq)
  return predicted_seq
  

#DATA LOADER

In [9]:

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 512

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader


def dataL(P, BATCH_SIZE, shuffle_bool):
    iter = DataLoader(P, batch_size=BATCH_SIZE, shuffle=shuffle_bool)

    return iter



train_iter = dataL(train_process, BATCH_SIZE, False)
valid_iter = dataL(val_process, BATCH_SIZE, False)
test_iter = dataL(test_process, BATCH_SIZE, False)

print(len(train_iter))
print(len(test_iter))
print(len(valid_iter))

100
8
8


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import string
import random
from collections import Counter
# Set random seed for reproducibility
SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)
import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
#os.environ['TORCH_USE_CUDA_DSA'] = '1'


In [11]:
!pip install --upgrade wandb
!wandb login f6b40fc0bcc13c1d6117718b14ef1ddd1c68a700 #my API key for wandb login 
import wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 49.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.16.6
    Uninstalling wandb-0.16.6:
      Successfully uninstalled wandb-0.16.6
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


#ATTENTION MODEL

In [12]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor


class Encoder(nn.Module):
    
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super(Encoder, self).__init__()
        l = [input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout]
        self.input_dim = l[0]
        self.emb_dim = l[1]
        self.enc_hid_dim = l[2]
        self.dec_hid_dim = l[3]
        self.dropout = l[4]

        self.embedding = nn.Embedding(l[0], l[1])
        bidir_bool = True   # Always True
        # GRU 
        self.rnn = nn.GRU(l[1], l[2], bidirectional=bidir_bool)
        temp = enc_hid_dim * 2
        self.fc = nn.Linear(temp, l[3])
        self.dropout = nn.Dropout(l[4])

    

    
    def forward(self, src: Tensor) -> Tuple[Tensor]:
        # Transpose the source tensor
        src_transposed = src.transpose(0, 1)

        # Embed the transposed source tensor with dropout
        embedded = self.dropout(self.embedding(src_transposed))

        # Pass the embedded tensor through the RNN
        outputs, hidden = self.rnn(embedded)

        # Concatenate and apply tanh activation to the last two layers of hidden states
        last_hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        hidden_combined = torch.tanh(self.fc(last_hidden))

        return outputs, hidden_combined

class Attention(nn.Module):
    

    def __init__(self, enc_hid_dim: int, dec_hid_dim: int, attn_dim: int):
        super(Attention, self).__init__()
        hid_l = [enc_hid_dim, dec_hid_dim]
        self.enc_hid_dim = hid_l[0]
        self.dec_hid_dim = hid_l[1]
        enc_hid_dim_double = 2 * enc_hid_dim
        temp = enc_hid_dim_double + dec_hid_dim
        self.attn_in = temp

        self.attn = nn.Linear(self.attn_in, attn_dim)
        
        
    def forward(self, decoder_hidden: Tensor, encoder_outputs: Tensor) -> Tensor:
        src_len = encoder_outputs.size(0)
        
        decoder_hidden_expanded = decoder_hidden.unsqueeze(1)

        repeated_decoder_hidden = decoder_hidden_expanded.repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        concat_hidden_outputs = torch.cat((repeated_decoder_hidden, encoder_outputs), dim=2)
        linear_transform = self.attn(concat_hidden_outputs)
        energy = torch.tanh(linear_transform)
        attention = torch.sum(energy, dim=2)

        attention_weights = F.softmax(attention, dim=1)

        return attention_weights
    
    
class Decoder(nn.Module):
    
    def __init__(self, output_dim: int, emb_dim: int, enc_hid_dim: int,
                 dec_hid_dim: int, dropout: float, attention: nn.Module):
        super().__init__()
        cons_l = [output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention]
        self.output_dim = cons_l[0]
        self.emb_dim = cons_l[1]
        self.enc_hid_dim =cons_l[2] 
        self.dec_hid_dim = cons_l[3]
        self.dropout_p = cons_l[4]
        self.attention = cons_l[5]

        self.embedding = nn.Embedding(cons_l[0], cons_l[1])
        self.rnn_input_size = cons_l[2] * 2 + cons_l[1]
        self.rnn = nn.GRU(self.rnn_input_size, cons_l[3])
        self.out_input_size = self.attention.attn_in + cons_l[1]
        self.out = nn.Linear(self.out_input_size, cons_l[0])
        self.dropout = nn.Dropout(cons_l[4])
    


    def _weighted_encoder_rep(self, decoder_hidden: Tensor, encoder_outputs: Tensor) -> Tensor:
        # Calculate attention weights
        attention_weights = self.attention(decoder_hidden, encoder_outputs)
        attention_weights = attention_weights.unsqueeze(1)  # Add a dimension for batch multiplication

        # Transpose encoder outputs for correct alignment
        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        # Calculate weighted encoder representation
        weighted_encoder_rep = torch.bmm(attention_weights, encoder_outputs)
        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)  # Transpose back for output

        return weighted_encoder_rep
    
    def forward(self, input: Tensor, decoder_hidden: Tensor, encoder_outputs: Tensor) -> Tuple[Tensor]:
        # Add a batch dimension to input
        input = input.unsqueeze(0)

        # Transpose input tensor
        input = input.permute(1, 0)

        # Apply embedding to the input tensor
        embedded_input = self.embedding(input)

        # Apply dropout to the embedded input tensor
        embedded = self.dropout(embedded_input)

        # Calculate the weighted encoder representation
        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden, encoder_outputs)

        # Transpose embedded tensor for concatenation
        embedded = embedded.permute(1, 0, 2)

        # Concatenate embedded tensor and weighted encoder representation
        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim=2)

        # Pass through the GRU layer
        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        # Squeeze the tensors to remove batch dimension
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        # Concatenate output, weighted encoder representation, and embedded tensor
        concatenated_output = torch.cat((output, weighted_encoder_rep, embedded), dim=1)

        # Pass through the output layer
        final_output = self.out(concatenated_output)

        return final_output, decoder_hidden.squeeze(0)
    


class Seq2Seq(nn.Module):
    def __init__(self,encoder: nn.Module,decoder: nn.Module,device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,src: Tensor,trg: Tensor,teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[0]
        max_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        #ATTENTION HEAT MAP METHOD
        #attentions = torch.zeros(max_len, batch_size, src.shape[1]).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        trg = trg.permute(1,0)
        output = trg[0,:]
        
        t = 1
        while t < max_len:
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            if random.random() < teacher_forcing_ratio:
                output = trg[t]
            else:
                output = output.max(1)[1]
            t += 1

        return outputs



def init_weights(m: nn.Module):
    params = iter(m.named_parameters())
    try:
        while True:
            name, param = next(params)
            if 'weight' in name:
                nn.init.normal_(param.data, mean=0, std=0.01)
            else:
                nn.init.constant_(param.data, 0)
    except StopIteration:
        pass





# ATTENTION WANDB IMPLEMENTATION

In [ ]:
import math
import time

sweep_config = {
            
            'method': 'random',
            'metric': { 'goal': 'maximize','name': 'Accuracy'},
            'parameters': 
                {
                    'num_epochs': {'values': [10]},
                    'cell_type': {'values': ['RNN', 'LSTM', 'GRU']},
                    'emb_dim': {'values': [128, 256, 512]},
                    'hidden_layer_dim': {'values': [128, 256, 512]},
                    'num_layers': {'values': [1, 2, 3]},
                    'dropout': {'values': [0.3, 0.5, 0.7]},
                    'attn_dim':{'values':[64,128,256]},
                    'optimizer' : {'values' : ['adam', 'rmsprop', 'adagrad']},
                    'learning_rate': {'values': [0.001, 0.005, 0.01, 0.1]},
                    'batch_size': {'values': [32, 64]},
                    'teacher_fr' : {'values': [0.3, 0.5, 0.7]},
                    'length_penalty' : {'values': [0.4, 0.5, 0.6]},
                    'bi_dir' : {'values': [True, False]},
                    'beam_width': {'values': [1, 2, 3, 4, 5]}
                }
            }
sweep_id = wandb.sweep(sweep_config, project="Deep_Learning_Seq2Seq_Transliteration_Assignment_3")
def train_model(model, train_iter, optimizer, criterion, device, CLIP):
    model.train()
    train_epoch_loss = 0
    train_epoch_acc = 0

    for _, (src, trg) in enumerate(train_iter):
        optimizer.zero_grad()
        src = src.to(device)
        trg = trg.to(device)

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg.permute(1, 0)
        trg = torch.reshape(trg[1:], (-1,))
        loss = criterion(output, trg)

        preds = torch.argmax(output, dim=1)
        non_pad_elements = (trg != 0).nonzero(as_tuple=True)[0]
        train_correct = preds[non_pad_elements] == trg[non_pad_elements]
        train_epoch_acc += train_correct.sum().item() / len(non_pad_elements)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)

        optimizer.step()

        train_epoch_loss += loss.item()

    train_epoch_loss /= len(train_iter)
    train_epoch_acc /= len(train_iter)

    return train_epoch_loss, train_epoch_acc

def evaluate_model(model, valid_iter, criterion, device):
    model.eval()
    val_epoch_loss = 0
    val_epoch_acc = 0

    with torch.no_grad():
        for _, (src, trg) in enumerate(valid_iter):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0)

            output = output[1:].view(-1, output.shape[-1])
            trg = trg.permute(1, 0)
            trg = torch.reshape(trg[1:], (-1,))

            loss = criterion(output, trg)

            val_epoch_loss += loss.item()

            preds = torch.argmax(output, dim=1)
            non_pad_elements = (trg != 0).nonzero(as_tuple=True)[0]
            val_correct = preds[non_pad_elements] == trg[non_pad_elements]
            val_epoch_acc += val_correct.sum().item() / len(non_pad_elements)

    val_epoch_loss /= len(valid_iter)
    val_epoch_acc /= len(valid_iter)

    return val_epoch_loss, val_epoch_acc

def run_training_loop(model, train_iter, valid_iter, optimizer, criterion, device, CLIP, N_EPOCHS):
    for epoch in range(N_EPOCHS):
        train_epoch_loss, train_epoch_acc = train_model(model, train_iter, optimizer, criterion, device, CLIP)
        val_epoch_loss, val_epoch_acc = evaluate_model(model, valid_iter, criterion, device)

        print(f'Epoch: {epoch+1:02} | Train accuracy: {train_epoch_acc*100:.3f} | Val accuracy: {val_epoch_acc*100:.3f}')
        wandb.log({"train_loss": train_epoch_loss, "train_accuracy": train_epoch_acc, "val_loss": val_epoch_loss, "val_accuracy": val_epoch_acc})

        if epoch == N_EPOCHS - 1:
            torch.cuda.empty_cache()


def sweep_train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {'emb_dim':256,'hidden_layer_dim':256,'attn_dim':128,'dropout':0.6,}
    # Initialize a new wandb run
    wandb.init(project='Deep_Learning_Seq2Seq_Transliteration_Assignment_3',config=config_defaults)
    wandb.run.name = (
        'Q2_c:' + str(wandb.config.cell_type) +
        '_e' + str(wandb.config.num_epochs) +
        '_es:' + str(wandb.config.emb_dim) +
        '_hs:' + str(wandb.config.hidden_layer_dim) +
        '_nle:' + str(wandb.config.num_layers) +
        '_nld:' + str(wandb.config.num_layers) +
        '_o:' + str(wandb.config.optimizer) +
        '_lr:' + str(wandb.config.learning_rate) +
        '_bs:' + str(wandb.config.batch_size) +
        '_tf:' + str(wandb.config.teacher_fr) +
        '_lp:' + str(wandb.config.length_penalty) +
        '_b:' + str(wandb.config.bi_dir) +
        '_bw:' + str(wandb.config.beam_width)
    ) 

    emb_dim = wandb.config.emb_dim
    hidden_layer_dim = wandb.config.hidden_layer_dim
    attn_dim = wandb.config.attn_dim
    dropout = wandb.config.dropout

    token_map_list_length = [len(eng_token_map), len(mar_token_map)]  
    INPUT_DIM = token_map_list_length[0]
    OUTPUT_DIM = token_map_list_length[1]
    enc_param = [INPUT_DIM, emb_dim, hidden_layer_dim, hidden_layer_dim, dropout]
    enc = Encoder(enc_param[0], enc_param[1],enc_param[2],enc_param[3],enc_param[4])
    att_param = [hidden_layer_dim, hidden_layer_dim, attn_dim]
    attn = Attention(att_param[0], att_param[1], att_param[2])
    dec_param = [OUTPUT_DIM, emb_dim, hidden_layer_dim, hidden_layer_dim, dropout, attn]
    dec = Decoder(dec_param[0],dec_param[1],dec_param[2],dec_param[3],dec_param[4],dec_param[5])

    model = Seq2Seq(enc, dec, device).to(device)
    model.apply(init_weights)
    opti = wandb.config.optimizer
    if opti == "adam":
        optimizer = optim.Adam(model.parameters())
    if opti == "rmsprop":
        optimizer = optim.RMSprop(model.parameters())
    if opti == "adagrad":
        optimizer = optim.Adagrad(model.parameters())
    else:
        optimizer = optim.Adam(model.parameters())  

    criterion = nn.CrossEntropyLoss()

    model.train()
    train_epoch_loss = 0
    val_epoch_loss,val_epoch_acc = 0,0
    train_epoch_acc = 0
    

    N_EPOCHS = wandb.config.num_epochs
    CLIP = 1


    # Example usage:
    run_training_loop(model, train_iter, valid_iter, optimizer, criterion, device, CLIP, N_EPOCHS)

#RUNNING THE SWEEP
wandb.agent(sweep_id, function=sweep_train, count=120)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: soltfjhz
Sweep URL: https://wandb.ai/cs23m074yash/Deep_Learning_Seq2Seq_Transliteration_Assignment_3/sweeps/soltfjhz
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


wandb: Agent Starting Run: n5y12tpl with config:
wandb: 	attn_dim: 256
wandb: 	batch_size: 64
wandb: 	beam_width: 4
wandb: 	bi_dir: True
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.7
wandb: 	emb_dim: 256
wandb: 	hidden_layer_dim: 256
wandb: 	learning_rate: 0.1
wandb: 	length_penalty: 0.6
wandb: 	num_epochs: 10
wandb: 	num_layers: 3
wandb: 	optimizer: rmsprop
wandb: 	teacher_fr: 0.5
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


In [14]:
config_defaults = {'emb_dim':256,'hidden_layer_dim':256,'attn_dim':128,'dropout':0.6,}
    
emb_dim = 256
hidden_layer_dim = 256
attn_dim = 128
dropout = 0.6

token_map_list_length = [len(eng_token_map), len(mar_token_map)]  
INPUT_DIM = token_map_list_length[0]
OUTPUT_DIM = token_map_list_length[1]
enc_param = [INPUT_DIM, emb_dim, hidden_layer_dim, hidden_layer_dim, dropout]
enc = Encoder(enc_param[0], enc_param[1],enc_param[2],enc_param[3],enc_param[4])
att_param = [hidden_layer_dim, hidden_layer_dim, attn_dim]
attn = Attention(att_param[0], att_param[1], att_param[2])
dec_param = [OUTPUT_DIM, emb_dim, hidden_layer_dim, hidden_layer_dim, dropout, attn]
dec = Decoder(dec_param[0],dec_param[1],dec_param[2],dec_param[3],dec_param[4],dec_param[5])

model = Seq2Seq(enc, dec, device).to(device)
model.apply(init_weights)
opti = "adam"
if opti == "adam":
    optimizer = optim.Adam(model.parameters())
if opti == "rmsprop":
    optimizer = optim.RMSprop(model.parameters())
if opti == "adagrad":
    optimizer = optim.Adagrad(model.parameters())
else:
    optimizer = optim.Adam(model.parameters())  

criterion = nn.CrossEntropyLoss()

model.train()

train_epoch_loss = 0
train_epoch_acc = 0
val_epoch_loss = 0
val_epoch_acc = 0

N_EPOCHS = 10
CLIP = 1
run_training_loop(model, train_iter, valid_iter, optimizer, criterion, device, CLIP, N_EPOCHS)

KeyboardInterrupt: 

Vanilla Seq2Seq


In [ ]:
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

import math
import time
import pandas as pd
import numpy as np
import random
from typing import Tuple
import torch
import torch.nn as nn
import torch.optim as optim
import string
import random
from collections import Counter
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader


# Data Load

In [ ]:
import pandas as pd

def load_data(path):
    data = pd.read_csv(path, header=None, names=["en", "ma", ""], skip_blank_lines=True)
    data = data.dropna(subset=['en', 'ma'])
    return data[['en', 'ma']]

# Define file paths
file_paths = [
    "/kaggle/input/marathi-dataset/mar/mar_train.csv",
    "/kaggle/input/marathi-dataset/mar/mar_valid.csv",
    "/kaggle/input/marathi-dataset/mar/mar_test.csv"
]

# Load data for train, val, and test
train, val, test = [load_data(path) for path in file_paths]

# Display test data and lengths
# print(test)
test_en = test['en'].tolist()
test_ma = test['ma'].tolist()
# print(test_en)
# print(test_ma)
print(len(train))
print(len(test))
print(len(val))


In [ ]:
def unique_tokenize(data):
    english_tokens = set(''.join(data['en']))
    marathi_tokens = set(''.join(data['ma']))
    return sorted(list(marathi_tokens)), sorted(list(english_tokens))
marathi_tokens , english_tokens = unique_tokenize(train)


In [ ]:
#REWRITE AGAIN
def tokenize_map(hindi_tokens , english_tokens):
    english_token_map = dict([(ch,i+1) for i,ch in enumerate(english_tokens)])
    marathi_token_map = dict([(ch,i+1) for i,ch in enumerate(marathi_tokens)])
    reverse_marathi_token_map = dict([(i+1,ch) for i,ch in enumerate(marathi_tokens)])
    #Adding blank space

    marathi_token_map[" "] = 0
    english_token_map[" "] = 0
    #addin BOS and EOS token 
    marathi_token_map[';']=65
    marathi_token_map['.']=66
    english_token_map[';']=27
    english_token_map['.']=28
    marathi_token_map['<unk>']=64
    
    reverse_marathi_token_map[64]='<unk>'
    reverse_marathi_token_map[65]=';'
    reverse_marathi_token_map[66]='.'
    reverse_marathi_token_map[0]=''

    return marathi_token_map, english_token_map,reverse_marathi_token_map

mar_token_map, eng_token_map,reverse_marathi_token_map = tokenize_map(marathi_tokens , english_tokens)
print(mar_token_map)
print(reverse_marathi_token_map)
print(eng_token_map)

print('mar:',len(mar_token_map))
print('eng:',len(eng_token_map))

In [ ]:

x = test['en'].values
y = test['ma'].values

max_eng_len = 2+max([len(i) for i in x]) 
max_mar_len = 2+max([len(i) for i in y])
print(max_eng_len,max_mar_len)

# One hot Encode

In [ ]:

import torch
unknown_token=64
def process(data):
    x, y = data['en'].values, data['ma'].values

    # Add start and end tokens to sentences
    x = [';' + sentence + '.' for sentence in x]
    y = [';' + sentence + '.' for sentence in y]
    a = torch.zeros((len(x), max_eng_len), dtype=torch.int64)
    b = torch.zeros((len(y), max_eng_len), dtype=torch.int64)

    # Process data
    for i, (xx, yy) in enumerate(zip(x, y)):
        for j, ch in enumerate(xx):
            a[i, j] = eng_token_map.get(ch, unknown_token)

        for j, ch in enumerate(yy):
            b[i, j] = mar_token_map.get(ch, unknown_token)


    # Create list of tuples (input tensor, target tensor)
    data = [(a[i], b[i]) for i in range(len(x))]

    return data


train_process=process(train)
val_process=process(val)
test_process=process(test)#print(train_process.shape)
print('\n')
print('num of rows:',len(train_process))
print('num of columns:',len(train_process[0]))
print(train_process[0][1])

In [ ]:

def reverse_tokenize(data):
    data = list(map(int, data))
    predicted_seq = ''.join(map(lambda idx: reverse_marathi_token_map[idx], data))
    return predicted_seq

In [ ]:

BATCH_SIZE = 16


def dataL(P, BATCH_SIZE, shuffle_bool):
    iter = DataLoader(P, batch_size=BATCH_SIZE, shuffle=shuffle_bool)

    return iter



train_iter = dataL(train_process, BATCH_SIZE, False)
valid_iter = dataL(val_process, BATCH_SIZE, False)
test_iter = dataL(test_process, BATCH_SIZE, False)

print(len(train_iter))
print(len(test_iter))
print(len(valid_iter))

# Wandb login

In [ ]:

!wandb login f6b40fc0bcc13c1d6117718b14ef1ddd1c68a700 #my API key for wandb login 
import wandb

# Vanilla encoder and decoder

In [ ]:
# Define the vocabulary of English and Devanagari characters

# Define the maximum sequence lengths for input and output sequences
MAX_LEN_EN = 30
MAX_LEN_MA = 30

# Define the start and end of sequence tokens
BOS_token = 28
EOS_token = 29


class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers=1 , cell_type="lstm", p=0.5):
        super().__init__()
        eL = [input_size, embedding_size, hidden_size, num_layers , cell_type, p]
        self.embedding = nn.Embedding(eL[0],eL[1])
        self.hidden_size = eL[2]
        self.num_layers = eL[3]
        self.cell_type = eL[4]
        self.dropout=nn.Dropout(eL[5])
        prob = p
        if cell_type.upper() == "GRU":
            self.rnn = nn.GRU(input_size, hidden_size, num_layers,dropout=prob)
        if cell_type.upper() == "LSTM":
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers,dropout=prob)
        if cell_type.upper() != "RNN":
            self.rnn = nn.GRU(input_size, hidden_size, num_layers,dropout=prob)
        else:
            self.rnn = nn.RNN(input_size, hidden_size, num_layers,dropout=prob)
    def forward(self, x):
        
        x = x.permute(1,0)
        pre_embedding = self.embedding(x)
        embedding=self.dropout(pre_embedding)
        if self.cell_type != "lstm":
            hidden, cell = self.rnn(embedding)
        elif self.cell_type == "lstm":
            outputs,(hidden,cell)=self.rnn(embedding)
        return hidden, cell


class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1, cell_type="lstm",p=0.5):
        super().__init__()
        dL = [input_size, embedding_size, hidden_size, output_size, num_layers, cell_type,p]
        self.embedding=nn.Embedding(dL[0], dL[1])
        self.hidden_size = dL[2]
        self.cell_type=dL[5]
        self.num_layers = dL[4]
        self.dropout=nn.Dropout(dL[6])

        self.output_size = dL[3]
        if cell_type.upper() == "GRU":
            self.rnn = nn.GRU(output_size, hidden_size, num_layers)
        if cell_type.lower() != "lstm":
            self.rnn = nn.RNN(output_size, hidden_size, num_layers)
        if cell_type.upper() == "RNN":
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers,dropout=p)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden,cell):
       
       x = x.unsqueeze(0)
       pre_embeddingD = self.embedding(x) 
       embedding= self.dropout(pre_embeddingD)
       
       if self.cell_type!="lstm":
         outputs, hidden = self.rnn(embedding, hidden)
       else:
         outputs,(hidden,cell) = self.rnn(embedding, (hidden,cell))
       

       predictions=self.fc(outputs)
       
       tempD = 0
       predictions = predictions.squeeze(tempD)
       
       if self.cell_type != "lstm":
         return predictions, hidden
       else:
         return predictions, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder,cell_type="lstm"):
        sL = [encoder,decoder,cell_type]
        super().__init__()
        self.encoder = sL[0]
        self.decoder = sL[1]
        self.cell_type = sL[2]

    def forward(self, source, target, teacher_force_ratio=0.5):
        sFL = [source, target, teacher_force_ratio]
        batch_size = sFL[0].shape[0]
        
        target_len = sFL[1].shape[1]
        target_vocab_size = self.decoder.output_size

        outputs = torch.zeros(target_len,batch_size,target_vocab_size).to(device)
        
        cell, hidden = self.encoder(sFL[0])
        
        target = sFL[1].permute(1,0)
        x = target[0,:]
        t = 1
        while t < target_len:
          if self.cell_type!="lstm":
            output, hidden = self.decoder(x,hidden,cell)
          else:
           output,hidden,cell =self.decoder(x,hidden,cell)

          outputs[t] = output
          
          best_guess = output.argmax(1)
          if random.random() >= teacher_force_ratio:
            x = best_guess  
          else:
            x = target[t]
          t += 1
        return outputs



In [ ]:

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs=10
learning_rate=0.001
len_token_map = [len(eng_token_map), len(mar_token_map)]
input_size_encoder = len_token_map[0]
input_size_decoder= len_token_map[1]
output_size= len_token_map[1]
max_size_E_D = [29, 67]
encoder_embedding_size = max_size_E_D[0]
decoder_embedding_size = max_size_E_D[1]

cell_type="gru"
hidden_size=256
num_layers=4
enc_dropout=0.2
dec_dropout=0.2
train_L = [input_size_encoder, encoder_embedding_size, hidden_size, num_layers , cell_type]
encoder_net=Encoder(len_token_map[0],max_size_E_D[0],train_L[2],train_L[3],train_L[4],p=enc_dropout).to(device)
decoder_net=Decoder(len_token_map[1],max_size_E_D[1],train_L[2],len_token_map[1],train_L[3],train_L[4],p=dec_dropout).to(device)
model_L = [encoder_net,decoder_net,cell_type]
model = Seq2Seq(model_L[0],model_L[1],model_L[2]).to(device)
criterion = nn.CrossEntropyLoss()


optimizer = optim.Adam(model.parameters(),lr=learning_rate)
def train(model: nn.Module, iterator: torch.utils.data.DataLoader, optimizer: optim.Optimizer, criterion: nn.Module, clip: float):


    epoch_loss, epoch_acc = 0, 0
    model.train()


    iterator_iter = iter(iterator)
    index = 0
    while True:
        try:
            src, trg = next(iterator_iter)
            optimizer.zero_grad()
            src = src.to(device)
            trg = trg.to(device)
            output = model(src, trg)

            output = output[1:].view(-1, output.shape[-1])
            trg = trg.permute(1,0)
            trg = torch.reshape(trg[1:], (-1,))
            loss = criterion(output, trg)

            # calculate accuracy
            preds = torch.argmax(output, dim=1)

            non_zero_elements = (trg != 0)
            non_zero_indices = non_zero_elements.nonzero(as_tuple=True)
            non_pad_elements = non_zero_indices[0]


            preds_non_pad = preds[non_pad_elements]
            trg_non_pad = trg[non_pad_elements]
            correct = preds_non_pad == trg_non_pad

            correct_sum = correct.sum().item()
            non_pad_length = len(non_pad_elements)
            accuracy = correct_sum / non_pad_length
            epoch_acc = epoch_acc + accuracy

            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

            optimizer.step()

            epoch_loss = epoch_loss + loss.item()
            index += 1
        except StopIteration:
            break
    loss_actual = epoch_loss / len(iterator)
    accuracy_actual = epoch_acc / len(iterator)
    return  loss_actual, accuracy_actual


def evaluate(model: nn.Module,iterator: torch.utils.data.DataLoader,criterion: nn.Module):
    epoch_loss, epoch_acc = 0, 0

    solution=[]
    model.eval()
    
    with torch.no_grad():
        iterator_iter = iter(iterator)
        
        index = 0
        while True:
          try:
            src, trg = next(iterator_iter)
            src = src.to(device)
            trg = trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing
            output = output[1:].view(-1, output.shape[-1])
            
            trg = trg.permute(1,0)
            trg = torch.reshape(trg[1:], (-1,))

            loss = criterion(output, trg)

            epoch_loss = epoch_loss + loss.item()
            # calculate accuracy
            preds = torch.argmax(output, dim=1)
            matrix_size = [16, 29]
            b=np.zeros((matrix_size[0],matrix_size[1]))
            i = 0
            while i < 16:
                j = 0
                while j < 29:
                    b[i][j] = preds[16 * j + i]
                    j += 1
                i += 1

            i = 0
            while i < 16:
              solution.append(reverse_tokenize(b[i]))
              i+=1

            non_zero_elements = trg != 0
            non_zero_indices = non_zero_elements.nonzero(as_tuple=True)
            non_pad_elements = non_zero_indices[0]


            preds_non_pad = preds[non_pad_elements]
            trg_non_pad = trg[non_pad_elements]
            correct = preds_non_pad == trg_non_pad
            

            correct_sum = correct.sum().item()
            non_pad_length = len(non_pad_elements)
            accuracy = correct_sum / non_pad_length
            epoch_acc += accuracy
            index += 1
          except StopIteration:
            break  
    loss_actual = epoch_loss / len(iterator)
    accuracy_actual = epoch_acc / len(iterator)
    return  loss_actual, accuracy_actual,solution




CLIP = 1
N_EPOCHS = 10


best_valid_loss = float('inf')
epoch = 0
while epoch < N_EPOCHS:
    train_loss, train_acc = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss, val_acc, solution = evaluate(model, valid_iter, criterion)
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train accuracy: {train_acc*33:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} | Val accuracy: {val_acc*33:.3f}')
    epoch+=1


test_loss,test_accuracy,solution_test = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test accuracy: {test_accuracy*33:7.3f} |')



# TEXT Prediction file


In [ ]:
file_path = '/kaggle/input/marathi-dataset/mar/test_dataset_predictions.txt'

with open(file_path, 'w') as file:
    file.write('Correct word\tPredicted word\n')

    # Write the data rows
    for en, ma, pred in zip(test_en, test_ma, solution_test):
        file.write(f'{ma}\t{pred}\n')
